In [13]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 27 18:55:35 2019

@author: JM
"""

import os
import re
from tqdm import tqdm
from tkinter.filedialog import askopenfilename 
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter,resolve1
from pdfminer.pdfdevice import PDFDevice, TagExtractor
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.cmapdb import CMapDB
from pdfminer.layout import LAParams
from pdfminer.image import ImageWriter


def isExistFile(path):
    file_name = path.split('/')[-1]
    
    for i in os.listdir("."):
        if file_name == i:
            return True
        
    return False

def pdf2txt(input_path):
    '''
    input_path : str, PDF File
    
    =============================
    
    return : str, text File path
    '''
    
    # input
    password=''
    pagenos=set()
    maxpages=0
    
    # output
    imagewriter = None
    rotation = 0
    codec = 'UTF-8'
    pageno = 1
    scale = 1
    caching = True
    showpageno = True
    laparams = LAParams()
    
    infp = open(input_path,"rb")
    
    output_path = input_path[:-4] + '_trans.txt'
    
    outfp = open(output_path,"w",encoding='UTF8')
    
    #page total num
    parser = PDFParser(infp)
    document = PDFDocument(parser)
    page_total_num = resolve1(document.catalog['Pages'])['Count']
    
    #
    rsrcmgr = PDFResourceManager(caching=caching)

    # pdf -> text converter
    device = TextConverter(rsrcmgr,
                           outfp,
                           codec=codec,
                           laparams=laparams, 
                           imagewriter=imagewriter)

    # pdf -> text interpreter
    interpreter = PDFPageInterpreter(rsrcmgr,device)
    
    # pdf -> text start
    with tqdm(total=page_total_num) as pbar:
        for page in PDFPage.get_pages(infp,
                                      pagenos,
                                      maxpages,
                                      password=password,
                                      caching=caching,
                                      check_extractable=True):

            page.rotate = (page.rotate+rotation) % 360     
            interpreter.process_page(page)
            
            pbar.update(1)

    print('==== success ====')

    outfp.close()
    infp.close()
    
    return output_path
    
def clean_text(path):
    '''
    path : str, text File Path
    
    
    ===========================
    
    return : list, sentences
    '''
    
    f = open(path,"rb")
    line_list = []
    
    while True:
        line = f.readline()
        line_list.append(line)
        if not line: break
    
    # remove nextline
    word = b" ".join(line_list).split()
    sentences = b" ".join(word)
    
    
    # remove ASCII
    # define pattern 
    pattern = re.compile(b"[\x80-\xff]")
    sentences = re.sub(pattern,b"",sentences)
    
    sentences = sentences.split(b". ")
    
    cleaned_txt = []
    
    for sentence in sentences:
        sentence = sentence.replace(b"- ",b'')
        sentence = sentence.replace(b"-",b'')
        cleaned_txt.append(sentence)
    
    f.close()
    
    os.remove(path)
    
    return cleaned_txt

txt_path = pdf2txt('C:/Users/JM/Desktop/MCNN.pdf')
cleaned_text = clean_text(txt_path)
print(cleaned_text)

100%|█████████████████████████████████████████| 9/9 [00:02<00:00,  3.42it/s]


==== success ====


[b'2016 IEEE Conference on Computer Vision and Pattern Recognition SingleImage Crowd Counting via MultiColumn Convolutional Neural Network Yingying Zhang Desen Zhou Siqin Chen Shenghua Gao Yi Ma {zhangyy2,zhouds,chensq,gaoshh,mayi}@shanghaitech.edu.cn Shanghaitech University Abstract This paper aims to develop a method than can accurately estimate the crowd count from an individual image with arbitrary crowd density and arbitrary perspective', b'To this end, we have proposed a simple but effective Multicolumn Convolutional Neural Network (MCNN) architecture to map the image to its crowd density map', b'The proposed MCNN allows the input image to be of arbitrary size or resolution', b'By utilizing lters with receptive elds of different sizes, the features learned by each column CNN are adaptive to variations in people/head size due to perspective effect or image resolution', b'Furthermore, the true density map is computed accurately based on geometryadaptive kernels which do not need kn

In [14]:
from konlpy.tag import Komoran
komoran = Komoran()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Program Files\\Java\\jre7\\bin\\client\\jvm.dll'